# ##############################################
# ASSIGNMENT 4
# NAME: Hajer Amr
# ID: 2305197
# TASK: Build CNN on Fashion-MNIST using Keras & PyTorch
# ##############################################


# ##############################################
# PART 1 — KERAS IMPLEMENTATION
# ##############################################

In [1]:
# 1️⃣ Imports
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
import matplotlib.pyplot as plt

In [2]:
# 2️⃣ Load Fashion-MNIST
(train_X, train_y), (test_X, test_y) = datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# 3️⃣ Preprocessing
train_X = train_X.reshape(-1, 28, 28, 1) / 255.0
test_X = test_X.reshape(-1, 28, 28, 1) / 255.0

In [4]:
# 4️⃣ Build CNN Model
keras_cnn = models.Sequential([
layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
layers.MaxPooling2D(2,2),


layers.Conv2D(64, (3,3), activation='relu'),
layers.MaxPooling2D(2,2),


layers.Flatten(),
layers.Dense(128, activation='relu'),
layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# 5️⃣ Compile
keras_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
# 6️⃣ Train
keras_history = keras_cnn.fit(train_X, train_y, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 55s 31ms/step - accuracy: 0.7640 - loss: 0.6503 - val_accuracy: 0.8715 - val_loss: 0.3443
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 78s 29ms/step - accuracy: 0.8858 - loss: 0.3104 - val_accuracy: 0.8790 - val_loss: 0.3197
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 82s 30ms/step - accuracy: 0.9038 - loss: 0.2657 - val_accuracy: 0.8982 - val_loss: 0.2771
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 81s 29ms/step - accuracy: 0.9195 - loss: 0.2239 - val_accuracy: 0.9088 - val_loss: 0.2500
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 50s 29ms/step - accuracy: 0.9240 - loss: 0.2034 - val_accuracy: 0.9142 - val_loss: 0.2393


In [7]:
# 7️⃣ Evaluate
test_loss, test_acc = keras_cnn.evaluate(test_X, test_y)
print("Keras Test Accuracy =", test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9104 - loss: 0.2579
Keras Test Accuracy = 0.9120000004768372


###############################################
# PART 2 — PYTORCH IMPLEMENTATION
###############################################

In [8]:
# 1️⃣ Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [9]:
# 2️⃣ Transformations
transform = transforms.Compose([ transforms.ToTensor() ])

In [10]:
# 3️⃣ Load Dataset
train_set = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_set = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)


train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.4MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.92MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 18.8MB/s]


In [12]:
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)


        self.fc1 = nn.Linear(64*5*5, 128)
        self.fc2 = nn.Linear(128, 10)


    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64*5*5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
# 5️⃣ Initialize Model, Loss, and Optimizer
model = FashionCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 6️⃣ Training Loop
for epoch in range(5):
    for images, labels in train_loader:
        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} completed")

Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed


In [17]:
# 7️⃣ Evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        out = model(images)
        _, predicted = torch.max(out, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print("PyTorch Test Accuracy =", correct/total)

PyTorch Test Accuracy = 0.8993
